In [1]:
from pathlib import Path
from todo_issues.parse import parse_file

In [7]:
parse_file(Path("example/example/main.py"))

[TodoItem(scope=None, content='module docstring\n"""', file=PosixPath('example/example/main.py'), token=TokenInfo(type=3 (STRING), string='"""Example Repo\n\nTODO: module docstring\n"""', start=(1, 0), end=(4, 3), line='"""Example Repo\n\nTODO: module docstring\n"""\n')),
 TodoItem(scope='CI', content='comment', file=PosixPath('example/example/main.py'), token=TokenInfo(type=61 (COMMENT), string='# TODO(CI): comment', start=(6, 0), end=(6, 19), line='# TODO(CI): comment\n')),
 TodoItem(scope=None, content='multiline', file=PosixPath('example/example/main.py'), token=TokenInfo(type=61 (COMMENT), string='# TODO: multiline', start=(8, 0), end=(8, 17), line='# TODO: multiline\n')),
 TodoItem(scope=None, content='function docstring\n    """', file=PosixPath('example/example/main.py'), token=TokenInfo(type=3 (STRING), string='"""Foo example.\n\n    TODO: function docstring\n    """', start=(13, 4), end=(16, 7), line='    """Foo example.\n\n    TODO: function docstring\n    """\n')),
 TodoIte